In [1]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-24 12:17:33--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip’

dataset_tsmc2014.zi 100%[===================>]  24.36M  6.28MB/s    in 5.8s    

2024-05-24 12:17:39 (4.19 MB/s) - ‘dataset_tsmc2014.zip’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
   creating: dataset_tsmc2014/
  inflating: dataset_tsmc2014/dataset_TSMC2014_NYC.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_readme.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_TKY.txt  


In [2]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [3]:
NUMCHECKIN = 30000

df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
df = df.sample(NUMCHECKIN)
print(df)

        User ID                  Venue ID         Venue category ID  \
70847       532  4a78c5fff964a52063e61fe3  4bf58dd8d48988d124941735   
69077       540  4f178012e4b0f1e7ba77133e  4f2a25ac4b909258e854f55f   
126192      912  4c0c2b61bbc676b0a8b44cd5  4eb1d4dd4b900d56c88a45fd   
223069      521  4c3725790a71c9b60a3f3ec9  4f2a25ac4b909258e854f55f   
135755      245  4f5bcf37e4b09a78bfc29e19  4bf58dd8d48988d116941735   
...         ...                       ...                       ...   
202407       88  4abd4c73f964a520948920e3  4bf58dd8d48988d1cb941735   
55187       178  4aed1141f964a52001cd21e3  4bf58dd8d48988d145941735   
156462      231  408da280f964a520d8f21ee3  4bf58dd8d48988d1a8941735   
29763       166  4cb66b191b0af04de5ced025  4bf58dd8d48988d116941735   
120077      341  4ad24038f964a52068e020e3  4bf58dd8d48988d16f941735   

                 Venue category name   Latitude  Longitude  Timezone  \
70847                         Office  40.730818 -73.991385      -240   
690

In [4]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
df['Venue ID'] = le.fit_transform(df['Venue ID'])
df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
df.drop(['Venue category name'],axis=1,inplace=True)
print(df)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
70847       532      2353                 69  40.730818 -73.991385      -240   
69077       540     11703                352  40.687357 -73.803965      -240   
126192      912      6975                339  40.710399 -73.988538      -240   
223069      521      7352                352  40.753696 -73.988441      -300   
135755      245     12061                 43  40.723766 -73.979138      -240   
...         ...       ...                ...        ...        ...       ...   
202407       88      2987                229  40.731766 -74.000865      -300   
55187       178      3456                108  40.775053 -73.984469      -240   
156462      231       446                198  40.729446 -73.993660      -240   
29763       166      8350                 43  40.726522 -73.986174      -240   
120077      341      3162                150  40.750281 -73.992448      -240   

                              UTC time 

In [5]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [6]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [7]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
numvenues = len(venuesdf)
print(venuesdf)


        Venue ID   Latitude  Longitude  Venue category ID
70847       2353  40.730818 -73.991385                 69
69077      11703  40.687357 -73.803965                352
126192      6975  40.710399 -73.988538                339
223069      7352  40.753696 -73.988441                352
135755     12061  40.723766 -73.979138                 43
...          ...        ...        ...                ...
10255       3979  40.619985 -74.032528                214
428         9354  40.882154 -73.881672                 31
180037      9073  40.867138 -73.896744                308
156462       446  40.729446 -73.993660                198
120077      3162  40.750281 -73.992448                150

[13326 rows x 4 columns]


In [8]:
!pip3 install torch --index-url https://download.pytorch.org/whl/${CUDA}
import torch
print(torch.__version__)
print(torch.version.cuda)
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-2.2.0+cu121.html

Looking in indexes: https://download.pytorch.org/whl/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [9]:
import networkx as nx
venuesdict = venuesdf.to_dict('records')

i = 0
nodeIds = {}
node_attr = []

for el in venuesdict:
  #print(el['Venue ID'])
  nodeIds[el['Venue ID']] = i
  node_attr.append(el['Venue category ID'])
  i+=1

print(len(node_attr))


13326


In [10]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))
'''
G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])
'''
from tqdm import tqdm
import torch
from torch_geometric.data import Data

edge_index = []
edges_attr = []

for i in tqdm(range(len(venuesdict) - 1)):

   #if venuesdict[i]['Venue ID'] not in node_attr:
   #  node_attr.append(venuesdict[i]['Venue ID'])

   for j in range(i + 1, len(venuesdict)):
    #  if venuesdict[j]['Venue ID'] not in node_attr:
    #     node_attr.append(venuesdict[j]['Venue ID'])


      if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                    venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
        edge_index.append([nodeIds[venuesdict[i]['Venue ID']], nodeIds[venuesdict[j]['Venue ID']]])
        edges_attr.append(1)

spatG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(spatG)


100%|██████████| 13325/13325 [04:44<00:00, 46.91it/s] 


Data(x=[13326], edge_index=[2, 2152960], edge_attr=[2152960])


In [11]:
import pickle

#spatpG = G

pickle.dump(spatG,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

In [12]:
#nx.draw(G)
print(spatG)

Data(x=[13326], edge_index=[2, 2152960], edge_attr=[2152960])


In [13]:

#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

edge_index = []
edges_attr = []

usrloc = {}

#seqG = nx.Graph()

for _, row in df.iterrows():

  #if not seqG.has_node(row['Venue ID']):
  #  seqG.add_node(row['Venue ID'], category=row['Venue category ID'])

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    #seqG.add_edge(usrloc[row['User ID']],row['Venue ID'])
    edge_index.append([nodeIds[usrloc[row['User ID']]], nodeIds[row['Venue ID']]])
    edges_attr.append(1)
    usrloc[row['User ID']] = row['Venue ID']


seqG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(seqG)






Data(x=[13326], edge_index=[2, 28917], edge_attr=[28917])


In [14]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

Data(x=[13326], edge_index=[2, 28917], edge_attr=[28917])


In [15]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






<ipython-input-15-f85843e15d2c>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')


In [16]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

#week['workdays'] = []
#week['weekend'] = []


for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  #if 1 <= day <=4:
  #  week['workdays'].append([row['Venue ID'],row['Venue category ID']])
  #else:
  #  week['weekend'].append([row['Venue ID'],row['Venue category ID']])

  if day not in week:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    week[day] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    week[day].append([row['Venue ID'],row['Venue category ID']])






       Venue ID  Venue category ID  localHour
0             0                261       23.0
1             1                 47       23.0
2             2                 43       13.0
3             2                 56       16.5
4             3                 47       21.0
...         ...                ...        ...
13416     13321                352       18.0
13417     13322                275        0.0
13418     13323                 51        3.0
13419     13324                 69        7.0
13420     13325                372        4.0

[13421 rows x 3 columns]
       Venue ID  Venue category ID  localHour  localDayofWeek
0             0                261       23.0            5.00
1             1                 47       23.0            1.00
2             2                 43       13.0            5.00
3             2                 56       16.5            4.25
4             3                 47       21.0            5.00
...         ...                ...        ...     

In [17]:
#dayG = nx.Graph()
#hourG = nx.Graph()

edge_index = []
edges_attr = []

for el in hours:
   for u in hours[el]:

    #if not hourG.has_node(u[0]):
    #  hourG.add_node(u[0], category=u[1])

    for v in hours[el]:

      if u != v:

        #if not hourG.has_node(u[0]):
        #  hourG.add_node(v[0], category=v[1])

        #hourG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

hourG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(hourG)




Data(x=[13326], edge_index=[2, 10362660], edge_attr=[0])
Data(x=[13326], edge_index=[2, 10362660], edge_attr=[0])


In [ ]:
pickle.dump(seqG,open('hourG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

In [18]:
edge_index = []
edges_attr = []


for el in week:
   for u in week[el]:

    #if not dayG.has_node(u[0]):
    #  dayG.add_node(u[0], category=u[1])

    for v in week[el]:

      if u != v:

        #if not dayG.has_node(u[0]):
        #  dayG.add_node(v[0], category=v[1])

        #dayG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

dayG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(dayG)

Data(x=[13326], edge_index=[2, 27080014], edge_attr=[0])
Data(x=[13326], edge_index=[2, 27080014], edge_attr=[0])


In [ ]:
pickle.dump(seqG,open('dayG.pickle','wb'))

In [19]:
sequences = {}
count = {}

for _, row in df.iterrows():

  if row['User ID'] not in sequences:
    sequences[row['User ID']] = [[nodeIds[row['Venue ID']],row['localHour'],row['localDayofWeek']]]


  else:
    sequences[row['User ID']].append([nodeIds[row['Venue ID']],row['localHour'],row['localDayofWeek']])


  if nodeIds[row['Venue ID']] not in count:
    count[nodeIds[row['Venue ID']]] = 1
  else:
    count[nodeIds[row['Venue ID']]] += 1

print(sequences)
seq = []
next = []

for el in sequences:
  entry = [el]
  for el2 in sequences[el][len(sequences[el])-1]:
    entry.append(el2)
  for i in range(len(sequences[el])-1):
    entry.append(sequences[el][i][0])
  seq.append(torch.tensor(entry))

  target = torch.zeros(numvenues)
  target[sequences[el][len(sequences[el])-1][0]] = 1
  next.append(target)
  print(count[sequences[el][len(sequences[el])-1][0]])
  if count[sequences[el][len(sequences[el])-1][0]] < 2:

    seq.append(torch.tensor(entry))
    next.append(target)

seq=torch.nn.utils.rnn.pad_sequence(seq,batch_first=True)
#seq=torch.nn.utils.rnn.pad_sequence(seq)

print(seq.shape)
print(len(next))



{532: [[0, 9, 2], [0, 9, 4], [2906, 23, 5], [0, 9, 4], [0, 9, 3], [0, 9, 0], [1018, 12, 3], [2891, 10, 3], [2776, 9, 1], [6699, 20, 3], [0, 9, 2], [0, 9, 1]], 540: [[1, 11, 1], [104, 11, 6], [104, 0, 3], [1038, 15, 5], [104, 22, 1], [2058, 14, 4], [2070, 10, 4], [2323, 14, 6], [2404, 15, 4], [104, 8, 6], [2404, 2, 3], [1038, 0, 2], [639, 11, 1], [8056, 12, 4], [1038, 13, 1], [8617, 8, 3], [2404, 11, 0], [2323, 13, 3], [544, 12, 4], [2323, 2, 3], [2070, 9, 2], [1038, 11, 2], [11435, 8, 0], [2404, 0, 5], [2404, 23, 4], [2404, 17, 0], [2404, 13, 3], [2404, 23, 6], [2404, 1, 5], [13297, 9, 4]], 912: [[2, 12, 6], [397, 22, 3], [3674, 17, 4], [4717, 14, 5], [5827, 14, 5], [7077, 21, 0], [7077, 21, 2], [7077, 15, 2], [8930, 20, 6], [9122, 15, 6], [78, 0, 3], [792, 19, 5], [9834, 10, 6], [9977, 17, 1], [9834, 10, 6], [11137, 22, 1], [7077, 13, 4], [3674, 18, 2], [745, 16, 5], [4371, 9, 4], [2920, 16, 1], [12955, 9, 5], [4371, 9, 0]], 521: [[3, 8, 2], [3, 8, 1], [523, 16, 2], [840, 7, 3], [1228

In [20]:
!pip install pytorch-lightning
import pytorch_lightning as pl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 9.4 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from torch_geometric.data import InMemoryDataset, DataLoader


# Funzione per calcolare alcune caratteristiche per ogni grafo
def calculate_graph_features(graph):
    num_nodes = graph.num_nodes
    num_edges = graph.num_edges
    avg_degree = num_edges / num_nodes
    return [num_nodes, num_edges, avg_degree]

# Lista dei grafi
graphs = [spatG, seqG, hourG, dayG]

# Calcolare le caratteristiche per ogni grafo
features = np.array([calculate_graph_features(graph) for graph in graphs])

# Usare Isolation Forest per individuare gli outliers
clf = IsolationForest(contamination=0.1)  # La contaminazione indica la proporzione di outliers
outliers = clf.fit_predict(features)

# Selezionare solo i grafi che non sono outliers
cleaned_graphs = [graphs[i] for i in range(len(graphs)) if outliers[i] == 1]

# Assegnare una label predefinita ai grafi che non hanno etichette
for graph in cleaned_graphs:
    if not hasattr(graph, 'y') or graph.y is None:
        graph.y = torch.tensor([0])  # Assegna una label predefinita, ad esempio 0
    # Aggiungi caratteristiche dummy ai nodi se non presenti
    if not hasattr(graph, 'x') or graph.x is None:
        graph.x = torch.ones((graph.num_nodes, 1))  # Aggiungi una caratteristica dummy per ogni nodo


# Define your custom dataset
class CustomGraphDataset(InMemoryDataset):
    def __init__(self, root, graphs, transform=None, pre_transform=None):
        self.graphs = graphs
        super(CustomGraphDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = self.collate(self.graphs)

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data, slices = self.collate(self.graphs)
        torch.save((data, slices), self.processed_paths[0])

# Create a custom dataset
dataset = CustomGraphDataset(root='.', graphs=cleaned_graphs)

# Split the dataset into training, validation, and test sets
train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.2, stratify=[data.y for data in dataset])
train_idx, val_idx = train_test_split(train_idx, test_size=0.25, stratify=[dataset[i].y for i in train_idx])

train_dataset = dataset[train_idx]
val_dataset = dataset[val_idx]
test_dataset = dataset[test_idx]

batch_size = 32

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:

class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Model(nn.Module):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(Model, self).__init__()

        # User Embedding Layer
        self.user_embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=64)

        # Location Embedding Layers
        self.gcn1 = GCN(location_dim, 64)
        self.gcn2 = GCN(64, 64)
        self.gcn3 = GCN(64, 64)

        # Sequence Embedding Layer
        self.self_attn = nn.MultiheadAttention(embed_dim=sequence_dim, num_heads=8)

        # Multihead Attention Layer
        self.multihead_attn = nn.MultiheadAttention(embed_dim=256, num_heads=8)

        # Classifier Layers
        self.linear1 = nn.Linear(256, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, 1)

    def forward(self, user, location, sequence):
        user_embed = self.user_embedding(user)

        # Location Embeddings
        edge_index1 = torch_geometric.nn.pool.knn_graph(location, k=10)
        location_embed1 = self.gcn1(location, edge_index1)
        location_embed2 = self.gcn2(location_embed1, edge_index1)
        location_embed3 = self.gcn3(location_embed2, edge_index1)

        # Sequence Embedding
        sequence_embed, _ = self.self_attn(sequence, sequence, sequence)

        # Concatenate all embeddings
        concat_embed = torch.cat((user_embed, location_embed3, sequence_embed), dim=1)

        # Multihead Attention
        attn_output, _ = self.multihead_attn(concat_embed, concat_embed, concat_embed)

        # Classifier
        out = F.relu(self.linear1(attn_output))
        out = F.relu(self.linear2(out))
        out = self.linear3(out)

        return out

class LightningModel(pl.LightningModule):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(LightningModel, self).__init__()
        self.model = Model(input_dim, location_dim, sequence_dim)

    def forward(self, user, location, sequence):
        return self.model(user, location, sequence)

    def training_step(self, batch, batch_idx):
        x, y = batch
        user, location, sequence = x[0], x[1], x[2]
        y_pred = self.model(user, location, sequence)
        loss = F.mse_loss(y_pred, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Example of training the model
input_dim = len(nodeIds)
location_dim = 64
sequence_dim = 64
model = LightningModel(input_dim, location_dim, sequence_dim)

trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, trainloader)


In [21]:
from sklearn.model_selection import train_test_split

batchsize = 32


train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2,stratify=torch.stack(next))

train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)

trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)

valset = torch.utils.data.TensorDataset(val_x,val_y)
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)

testset = torch.utils.data.TensorDataset(test_x,test_y)
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
#import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Model(nn.Module):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(Model, self).__init__()

        # User Embedding Layer
        self.user_embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=64)

        # Location Embedding Layers
        self.gcn1 = GCN(location_dim, 64)
        self.gcn2 = GCN(64, 64)
        self.gcn3 = GCN(64, 64)

        # Sequence Embedding Layer
        self.self_attn = nn.MultiheadAttention(embed_dim=location_dim, num_heads=8)

        # Multihead Attention Layer
        self.multihead_attn = nn.MultiheadAttention(embed_dim=256, num_heads=8)

        # Classifier Layers
        self.linear1 = nn.Linear(256, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, 1)

    def forward(self, user, timestamp, location, sequence):
        user_embed = self.user_embedding(user)

        k = 10
        # Location Embeddings
        edge_index1 = torch_geometric.nn.pool.knn_graph(spatG.x,k)

        location_embed1 = self.gcn1(location, edge_index)
        location_embed2 = self.gcn2(location_embed1, edge_index)
        location_embed3 = self.gcn3(location_embed2, edge_index)

        # Sequence Embedding
        sequence_embed, _ = self.self_attn(sequence, sequence, sequence)

        # Concatenate all embeddings
        concat_embed = torch.cat((user_embed, location_embed3, sequence_embed), dim=1)

        # Multihead Attention
        attn_output, _ = self.multihead_attn(concat_embed, concat_embed, concat_embed)

        # Classifier
        out = F.relu(self.linear1(attn_output))
        out = F.relu(self.linear2(out))
        out = self.linear3(out)

        return out

# Define Lightning model using PyTorch Lightning
import pytorch_lightning as pl

class LightningModel(pl.LightningModule):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(LightningModel, self).__init__()
        self.model = Model(input_dim, location_dim, sequence_dim)

    def forward(self, user, timestamp, location, sequence):
        return self.model(user, timestamp, location, sequence)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.model(x['user'], x['timestamp'], x['location'], x['sequence'])
        loss = F.mse_loss(y_pred, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Example of training the model
input_dim = 100
location_dim = 64
sequence_dim = 64
model = LightningModel(input_dim, location_dim, sequence_dim)

trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_dataloader)